An enumeration is a set of symbolic names (members) bound to unique, constant values. Within an enumeration, the members can be compared by identity, and the enumeration itself can be iterated over.

As reported in https://stackoverflow.com/questions/702834/, enumerations can be creatded using a simple class, assigning attributes to a range

In [1]:
class Materials:
    Shaded, Shiny, Transparent, Matte = range(1, 5)

In [3]:
Materials.Shaded

1

# Module `enum`

The module provides 4 enumeration classes : `Enum`, `IntEnum`, `IntFlag`, `Flag`, a decorator `@unique` and a helper function `auto()`

enum are created using the `class` statement, subclassing `Enum`<br/>
enum values can be anything. If value is unimportant, use the module provided `enum.auto`

In [4]:
from enum import Enum, IntEnum, IntFlag, Flag, auto, unique

`Enum` members values can be anything: `int`, `str`, etc..  
`IntEnum` are strict `int`. A `ValueError` in raised if attempting to set anything else

In [5]:
class URL0(Enum):
    CULTURE = "http://www.franceculture.fr"
    INTER = "http://www.franceinter.fr"
    MUSIQUE = "https://www.francemusique.fr"
    INFO = "https://www.francetvinfo.fr"

In [6]:
URL0.CULTURE

<URL0.CULTURE: 'http://www.franceculture.fr'>

In [7]:
# Enum are iterable
for station in URL0:
    print(f"{station:12}", f"name: {station.name:8} / value: {station.value}", sep=" ->  ")

URL0.CULTURE ->  name: CULTURE  / value: http://www.franceculture.fr
URL0.INTER   ->  name: INTER    / value: http://www.franceinter.fr
URL0.MUSIQUE ->  name: MUSIQUE  / value: https://www.francemusique.fr
URL0.INFO    ->  name: INFO     / value: https://www.francetvinfo.fr


Call of an Enum passing by value returns the Enum element

In [8]:
URL0("http://www.franceculture.fr")

<URL0.CULTURE: 'http://www.franceculture.fr'>

## Enum derived class: `IntEnum`

With usage of `auto()` for automatic value completion

In [9]:
class URL1(IntEnum):
    CULTURE = 1
    INTER = 2
    MUSIQUE = auto()
    INFO = auto()

Comparison to int is granted

In [10]:
URL1.INFO

<URL1.INFO: 4>

In [11]:
URL1.INFO == 1

False

`IntEnum` can very easily recreated

In [12]:
IntEnum??

Init signature:
IntEnum(
    value,
    names=None,
    *,
    module=None,
    qualname=None,
    type=None,
    start=1,
)
Source:        
class IntEnum(int, Enum):
    """Enum where members are also (and must be) ints"""
File:           /opt/conda/lib/python3.7/enum.py
Type:           EnumMeta
Subclasses:     Signals, Handlers, Sigmasks, _ParameterKind, AddressFamily, SocketKind, _SSLMethod, AlertDescription, SSLErrorNumber, VerifyMode, ...


Subclassing an enumeration is allowed only if the enumeration does not define any members.

In [13]:
class StrEnum(str, Enum):
    """Enum where members are str"""

In [14]:
class URL2(StrEnum):
    CULTURE = "http://www.franceculture.fr"
    INTER = "http://www.franceinter.fr"
    MUSIQUE = "https://www.francemusique.fr"
    INFO = "https://www.francetvinfo.fr"

Direct comparison to `str`is possible

In [15]:
URL2.CULTURE == "http://www.franceculture.fr"

True

## Programmatic access to enumeration members and their attributes

In [16]:
class URL3(Enum):
    CULTURE = "http://www.franceculture.fr"
    INTER = "http://www.franceinter.fr"
    MUSIQUE = "https://www.francemusique.fr"
    INFO = "https://www.francetvinfo.fr"
    
    def __init__(self, *pargs, **kwargs):
        print(f"__init__ is ran with postional arguments {pargs} and keyword arguments {kwargs}")
        self.scheme = pargs[0].split(":")[0]
        
    def method(self):
        print(f"A method is called, applied on {self} with value {self.value}")
    
    @property
    def prop(self):
        print(f"A dynamic property, applied on {self} with value {self.value}")
    
    @classmethod
    def favorite_station(cls):
        # cls here is the enumeration
        return cls.CULTURE

__init__ is ran with postional arguments ('http://www.franceculture.fr',) and keyword arguments {}
__init__ is ran with postional arguments ('http://www.franceinter.fr',) and keyword arguments {}
__init__ is ran with postional arguments ('https://www.francemusique.fr',) and keyword arguments {}
__init__ is ran with postional arguments ('https://www.francetvinfo.fr',) and keyword arguments {}


In [17]:
URL3.CULTURE.scheme

'http'

In [18]:
URL3.CULTURE.method()

A method is called, applied on URL3.CULTURE with value http://www.franceculture.fr


In [19]:
URL3.CULTURE.prop

A dynamic property, applied on URL3.CULTURE with value http://www.franceculture.fr


In [20]:
URL3.favorite_station()

<URL3.CULTURE: 'http://www.franceculture.fr'>

Some more complex subclass with programmatic access to attributes

In [21]:
from urllib.parse import urlparse

class URLEnum(str, Enum):
    """Enum where members are also (and must be) URL"""
    def __init__(self, url):
        "scheme://netloc/path;parameters?query#fragment"
        for (attr, value) in zip("scheme netloc path parameters query fragment".split(" "),
                                 list(urlparse(url))):
            setattr(self, attr, value)
        assert self.scheme in ['http', 'https'], f"URL {self} provides neither http nor https scheme"

In [22]:
class URLEnum(str, Enum):
    def __new__(cls, value):
        obj = str.__new__(cls)
        obj._value_ = urlparse(value)
        return obj

In [23]:
class URL5(URLEnum):
    CULTURE = "http://www.franceculture.fr"
    INTER = "http://www.franceinter.fr"
    MUSIQUE = "https://www.francemusique.fr"
    INFO = "https://www.francetvinfo.fr"

In [24]:
URL5.CULTURE.value.scheme

'http'

In [25]:
URL5.CULTURE == "http://www.franceculture.fr"

False

## Flag

Flag have `__bool__`, `__or__`, `__and__`,  `__xorr__`, `__invert__`methods

In [26]:
class Color(Flag):
    RED = auto()
    BLUE = auto()
    GREEN = auto()

In [27]:
Color.RED

<Color.RED: 1>

In [28]:
Color.RED & Color.GREEN

<Color.0: 0>

In [29]:
bool(Color.RED & Color.GREEN)

False

## `IntFlag`

`IntFlag` subclasses `Int`and `Flag`. Members can be combined using the bitwise operators (&, |, ^, ~) and the result is still an `IntFlag`

In [30]:
class Perm(IntFlag):
    R = 4
    W = 2
    X = 1

In [31]:
Perm.R | Perm.W

<Perm.R|W: 6>

In [32]:
Perm.R + Perm.W

6

In [33]:
RW = Perm.R | Perm.W
Perm.R in RW

True

## Functional API

In [34]:
Enum?

Init signature: Enum(value, names=None, *, module=None, qualname=None, type=None, start=1)
Docstring:     
Generic enumeration.

Derive from this class to define new enumerations.
File:           /opt/conda/lib/python3.7/enum.py
Type:           EnumMeta
Subclasses:     IntEnum, Flag, Purpose, _SendfileMode, SelectionType, PasteMode, EditingMode, Priority, ColorDepth, MouseEventType, ...


In [35]:
URL6 = Enum('Radio', 'Culture Inter Musique Info', type=str, start=20)

In [36]:
URL6.Inter

<Radio.Inter: '21'>